In [578]:
import init_spark_env
import pyspark
import pandas as pd

In [579]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, avg, stddev, stddev_pop, isnan, lower, transform, udf, replace
from pyspark.sql.types import IntegerType

In [580]:
spark = SparkSession.builder.master('local').enableHiveSupport().appName('test_cli').getOrCreate()

In [581]:
spark

In [582]:
data = spark.read.csv('data/fifa_s2.csv')

In [583]:
data.show()

+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+--------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|  _c0|              _c1|_c2|                _c3|    _c4|      _c5|                _c6|  _c7| _c8|           _c9|                _c10|       _c11|    _c12|  _c13|                _c14|       _c15|  _c16|          _c17|
+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+--------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|   ID|             Name|Age|        Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Rep...|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
| 1179|        G. Buffon| 40|              Italy|     88|       88|Paris Saint-Germain| 4000|  77|         Right|               

In [584]:
parser = spark.read.option("header", "true").option("nullValue", "?").option("inferSchema", "true").csv("data/fifa_s2.csv")

In [585]:
parser.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Value: integer (nullable = true)
 |-- Wage: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Position: string (nullable = true)
 |-- Joined: integer (nullable = true)
 |-- Contract Valid Until: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Release Clause: double (nullable = true)



In [586]:
parser.dtypes

[('ID', 'int'),
 ('Name', 'string'),
 ('Age', 'int'),
 ('Nationality', 'string'),
 ('Overall', 'int'),
 ('Potential', 'int'),
 ('Club', 'string'),
 ('Value', 'int'),
 ('Wage', 'int'),
 ('Preferred Foot', 'string'),
 ('International Reputation', 'int'),
 ('Skill Moves', 'int'),
 ('Position', 'string'),
 ('Joined', 'int'),
 ('Contract Valid Until', 'string'),
 ('Height', 'double'),
 ('Weight', 'double'),
 ('Release Clause', 'double')]

In [587]:
parser.first()

Row(ID=1179, Name='G. Buffon', Age=40, Nationality='Italy', Overall=88, Potential=88, Club='Paris Saint-Germain', Value=4000, Wage=77, Preferred Foot='Right', International Reputation=4, Skill Moves=1, Position='GK', Joined=2018, Contract Valid Until='01/01/2019', Height=6.333333333, Weight=203.0, Release Clause=7400.0)

In [588]:
parser.count()

2399

In [589]:
parser.show(5)

+-----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|   ID|     Name|Age|Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+-----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
| 1179|G. Buffon| 40|      Italy|     88|       88|Paris Saint-Germain| 4000|  77|         Right|                       4|          1|      GK|  2018|          01/01/2019|6.333333333| 203.0|        7400.0|
| 5479| Casillas| 37|      Spain|     82|       82|           FC Porto| 1500|  10|          Left|                       4|          1|    NULL|  2015|          01/01/2019|6.083

In [590]:
parser.cache()

24/12/09 22:49:35 WARN CacheManager: Asked to cache already cached data.


DataFrame[ID: int, Name: string, Age: int, Nationality: string, Overall: int, Potential: int, Club: string, Value: int, Wage: int, Preferred Foot: string, International Reputation: int, Skill Moves: int, Position: string, Joined: int, Contract Valid Until: string, Height: double, Weight: double, Release Clause: double]

In [591]:
parser.summary().show()

+-------+------------------+--------------+------------------+-----------+-----------------+-----------------+--------------------+-----------------+------------------+--------------+------------------------+------------------+--------+-----------------+--------------------+-------------------+------------------+------------------+
|summary|                ID|          Name|               Age|Nationality|          Overall|        Potential|                Club|            Value|              Wage|Preferred Foot|International Reputation|       Skill Moves|Position|           Joined|Contract Valid Until|             Height|            Weight|    Release Clause|
+-------+------------------+--------------+------------------+-----------+-----------------+-----------------+--------------------+-----------------+------------------+--------------+------------------------+------------------+--------+-----------------+--------------------+-------------------+------------------+------------------

In [592]:
parser.describe().show()

+-------+------------------+--------------+------------------+-----------+-----------------+-----------------+--------------------+-----------------+------------------+--------------+------------------------+------------------+--------+-----------------+--------------------+-------------------+------------------+------------------+
|summary|                ID|          Name|               Age|Nationality|          Overall|        Potential|                Club|            Value|              Wage|Preferred Foot|International Reputation|       Skill Moves|Position|           Joined|Contract Valid Until|             Height|            Weight|    Release Clause|
+-------+------------------+--------------+------------------+-----------+-----------------+-----------------+--------------------+-----------------+------------------+--------------+------------------------+------------------+--------+-----------------+--------------------+-------------------+------------------+------------------

In [593]:
data = parser

In [594]:
data = data.withColumn("name", lower("Name"))
data = data.withColumn("nationality", lower("Nationality"))
data = data.withColumn("club", lower("Club"))
data = data.withColumn("preferred foot", lower("Preferred Foot"))
data = data.withColumn("position", lower("Position"))
data.count()

2399

In [595]:
data.toPandas().isna().sum()

ID                            0
name                          0
Age                           0
nationality                   7
Overall                       0
Potential                     0
club                        241
Value                       257
Wage                          0
preferred foot                0
International Reputation     10
Skill Moves                   6
position                     21
Joined                        0
Contract Valid Until        273
Height                        0
Weight                        0
Release Clause                3
dtype: int64

In [596]:
data.filter(data.nationality.isNull()).show()

+------+-------------+---+-----------+-------+---------+--------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|    ID|         name|Age|nationality|Overall|Potential|                club|Value|Wage|preferred foot|International Reputation|Skill Moves|position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+------+-------------+---+-----------+-------+---------+--------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|156432|    g. hoarau| 34|       NULL|     77|       77|      bsc young boys| 4700|  26|         right|                       2|          3|      ls|  2014|          01/01/2020|6.333333333| 187.0|        7000.0|
|186146|   d. welbeck| 27|       NULL|     77|       77|             arsenal| 9500|  95|         right|                       3|          3|      lw|  2

In [597]:
data = data.fillna(value='unknown', subset=['nationality','club','position', 'Contract Valid Until'])
data = data.fillna(value=0, subset=['Value'])

In [598]:
data.select('International Reputation').distinct().show()

+------------------------+
|International Reputation|
+------------------------+
|                       1|
|                       3|
|                       5|
|                       4|
|                       2|
|                    NULL|
+------------------------+



In [599]:
data = data.fillna(value=0, subset=['International Reputation'])

In [600]:
data.select('Skill Moves').distinct().show()

+-----------+
|Skill Moves|
+-----------+
|          1|
|          3|
|          5|
|          4|
|          2|
|       NULL|
+-----------+



In [601]:
data = data.fillna(value=0, subset=['Skill Moves'])

In [602]:
data.select('Release Clause').distinct().show()

+--------------+
|Release Clause|
+--------------+
|        4800.0|
|         596.0|
|   4585.060806|
|         934.0|
|         305.0|
|       56400.0|
|         810.0|
|       18400.0|
|         170.0|
|         720.0|
|       10300.0|
|         608.0|
|       60200.0|
|         735.0|
|       90000.0|
|          70.0|
|         878.0|
|       16800.0|
|         311.0|
|       12600.0|
+--------------+
only showing top 20 rows



In [603]:
data = data.fillna(value=0, subset=['Release Clause'])

In [604]:
data.toPandas().isna().sum()

ID                          0
name                        0
Age                         0
nationality                 0
Overall                     0
Potential                   0
club                        0
Value                       0
Wage                        0
preferred foot              0
International Reputation    0
Skill Moves                 0
position                    0
Joined                      0
Contract Valid Until        0
Height                      0
Weight                      0
Release Clause              0
dtype: int64

In [605]:
data.count()

2399

In [606]:
data = data.dropDuplicates()

In [607]:
data.count()

2397

In [608]:
data.select('club').distinct().count()

81

In [609]:
list_club = data.select('club').distinct().sort('club').rdd.flatMap(list).collect()

In [610]:
list_club

['1. fc heidenheim 1846',
 '1. fc kaiserslautern',
 '1. fc köln',
 '1. fc magdeburg',
 '1. fc nürnberg',
 '1. fc union berlin',
 '1. fsv mainz 05',
 'aalborg bk',
 'arsenal',
 'as béziers',
 'as monaco',
 'aston villa',
 'atalanta',
 'bayer 04 leverkusen',
 'borussia dortmund',
 'borussia mönchengladbach',
 'brescia',
 'brighton & hove albion',
 'brisbane roar',
 'bristol city',
 'bristol rovers',
 'bsc young boys',
 'burnley',
 'bursaspor',
 'burton albion',
 'bury',
 'ca osasuna',
 'cagliari',
 'cambridge united',
 'cardiff city',
 'carlisle united',
 'carpi',
 'cd antofagasta',
 'cd aves',
 'cd everton de viña del mar',
 'cd feirense',
 'cd huachipato',
 'cd leganés',
 'cd lugo',
 'cd nacional',
 'cd numancia',
 "cd o'higgins",
 'cd palestino',
 'cd tenerife',
 'cd tondela',
 'cd universidad de concepción',
 'ceará sporting club',
 'celtic',
 'central coast mariners',
 'cerezo osaka',
 'cf rayo majadahonda',
 'cf reus deportiu',
 'chamois niortais football club',
 'changchun yatai f

In [611]:
def group_age(age):
    if age<20:
        return 1
    elif 20 <= age <= 30:
        return 2
    elif 30 < age <=36:
        return 3
    elif age > 36:
        return 4

In [612]:
age_fun = udf(lambda x: group_age(x), IntegerType())

In [613]:
data = data.withColumn('age_grup', age_fun(col('Age')))

In [614]:
data.select('Age', 'age_grup').show(10)

+---+--------+
|Age|age_grup|
+---+--------+
| 33|       3|
| 32|       3|
| 28|       2|
| 28|       2|
| 25|       2|
| 23|       2|
| 18|       1|
| 20|       2|
| 23|       2|
| 35|       3|
+---+--------+
only showing top 10 rows



In [615]:
data.select('age_grup').groupBy('age_grup').count().sort('count').show()

+--------+-----+
|age_grup|count|
+--------+-----+
|       4|   15|
|       1|  270|
|       3|  339|
|       2| 1773|
+--------+-----+

